In [104]:
import numpy as np
import pandas as pd
from astroquery.mast import Catalogs
import astropy.units as u
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
import astropy.coordinates as coord
import os
from astropy.table import Table,vstack
from tqdm import tqdm
import warnings
import time

In [62]:
Simbad.add_votable_fields("otype","sp")

In [67]:
curls = []
for fn in os.listdir('ref'):
    with open(f"ref/{fn}",'r') as f:
        curls += f.readlines()[1:]
#print(curls)

In [71]:
tic_ids = []
for curl in curls[1:]:
    tic_ids += [int(curl.strip().split(' ')[-1].split('-')[2])]
print(f"There are {len(curls)} fast light curves available.")

There are 47055 fast light curves available.


In [72]:
tic = Catalogs.query_criteria(catalog='TIC',ID=tic_ids)

In [73]:
cat = pd.DataFrame({'ID':tic['ID'],
                    'ra':tic['ra'],
                    'dec':tic['dec']})#,
                    #'match_dist':np.zeros(len(tic)),
                    #'spectra_type':['']*len(tic)})

In [111]:
simbad = Table()
for i in tqdm(np.arange(0,len(cat),1000)): # if you submit too large of a bulk request, SIMBAD will time out
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        simbad_temp = Simbad.query_region(
            coord.SkyCoord( cat['ra'][i:i+1000].tolist(),
                           cat['dec'][i:i+1000].tolist(),
                   unit=(u.deg, u.deg)),
            radius=35/60/60*u.deg) # 35as search radius
    simbad = vstack([simbad,simbad_temp])
    time.sleep(10) # if you submit too many requests per minute, SIMBAD will blacklist you

100%|███████████████████████████████████████████| 24/24 [11:36<00:00, 29.03s/it]


In [112]:
simbad

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,OTYPE,PLX_VALUE,PLX_PREC,PLX_ERROR,PLX_QUAL,PLX_BIBCODE,SP_TYPE,OTYPE_2,SP_TYPE_2,OTYPE_3,SP_TYPE_3,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,,mas,,mas,,,,,,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,object,float64,int16,float32,str1,object,object,object,object,object,object,int32
BD+33 4813,00 00 04.0830,+34 11 18.792,14,14,0.157,0.127,90,A,O,2020yCat.1350....0G,SB*,20.194,3,0.225,A,2020yCat.1350....0G,G0,SB*,G0,SB*,G0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SDSSCGB 18665.3,12 53 52.2695,-00 10 32.217,14,14,39.687,17.750,90,C,O,2020yCat.1350....0G,Galaxy,--,0,--,,,,Galaxy,,Galaxy,,980


In [113]:
simbad_positions = coord.SkyCoord(simbad['RA'].tolist(),
                                  simbad['DEC'].tolist(),
                                  unit=(u.hourangle, u.deg))

In [114]:
tess_position = coord.SkyCoord(tic[0]['ra'],tic[0]['dec'],
                               unit = (u.deg,u.deg))

In [118]:
for i,skypos in enumerate(zip(ra,dec)):
    tess_position = coord.SkyCoord(skypos[0],skypos[1],
                                   unit = (u.deg,u.deg))
    match = tess_position.match_to_catalog_sky(simbad_positions)
    if match[1].deg>1e-05: # maximum radius for a valid match, ~0.36"
        cat.loc[i,'match_dist_as']=match[1].deg*60*60
        cat.loc[i,'spectral_type']='-'
        continue
    cat.loc[i,'match_dist_as']=match[1].deg*60*60
    cat.loc[i,'spectral_type']=simbad[match[0]]['SP_TYPE']

In [145]:
cat.to_csv('object_table.csv',index=None)

In [139]:
dwarf_curls=[]
n,m=0,0
for source in cat.iterrows():
    if (str(source[1]['spectral_type']).startswith('M') or
        str(source[1]['spectral_type']).startswith('K')):
        tic_id = source[1]['ID']
        n+=1
        for curl in curls:
            if f"{tic_id}-" in curl:
                dwarf_curls+=[curl]
                m+=1
print(n,m)

233 318


In [144]:
with open('ref/dwarf_curls.sh','w') as f:
    for c in dwarf_curls:
        f.write(c)